In [1]:
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
import os
import numpy as np

In [2]:
# implement 2D DCT
def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho') 

def dct_compression(img):
    imf = np.float32(img)
    comp = dct2(imf)/255. # Scale to force low values to 0
    comp = np.uint8(idct2(comp)*255)
    return comp

In [3]:
def cv_disp_img(img, title="image"):
    while True:
        cv2.imshow(title, img)
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # q key pressed so quit
            print("Quitting...")
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
# Generate Structure contiaining info on dataset
directory = "./Dataset/"



In [4]:
# Combine dataset of images into an accessible array
# Since images are already in grayscale, can import as single channel
num_images = len(os.listdir("./Dataset/"))
training_images = np.zeros([num_images,192,168])
ct = 0
for file in np.sort(os.listdir("./Dataset/")):
    if ".pgm" in file:
        img = cv2.imread(os.path.join("./Dataset",file),0)
        training_images[ct,:,:] = img
        ct += 1
training_images = np.uint8(training_images)

In [5]:
# DCT compress all images to increase number of 0's
dct_images = np.zeros([num_images,192,168])
for i in range(num_images):
    dct_images[i,:,:] = dct_compression(training_images[i])
dct_images = np.uint8(dct_images)

In [6]:
print(np.count_nonzero(training_images[456]==0),np.count_nonzero(dct_images[456]==0))

663 1432


In [7]:
cv_disp_img(cv2.hconcat((training_images[456],dct_images[456])))

Quitting...


In [ ]:
# 2DPCA
def give_P(eig_vals,quality):
    s = np.sum(eig_vals)
    thresh = s * quality/100
    t = 0
    P = 0
    while t < thresh:
        t += eig_vals[P]
        P += 1
        
    return P
        
def reduce_dim(images,mean_subtracted):
    num = images.shape[0]
    g_t = np.zeros((images.shape[2],images.shape[2]))
    for i in range(num):
        temp_gt = np.dot(mean_subtracted[i].T,mean_subtracted[i])
        g_t += temp_gt
    
    g_t /= num
    e_vals, e_vec = np.linalg.eig(g_t)
    P = give_P(eig_vals=e_vals,quality=95)
    print("P: ",P)
    new_bases = e_vec[:,0:P]
    new_cord = np.dot(images,new_bases.T)
    return new_cord
        

mean_face = np.mean(dct_images,0)
images_mean_subtracted = dct_images - mean_face




In [ ]:
new_images = reduce_dim(dct_images,images_mean_subtracted)

In [ ]:
new_images[456].shape

In [ ]:
cv_disp_img(cv2.hconcat((training_images[456],dct_images[456],new_images[456])))

Populating the interactive namespace from numpy and matplotlib


Released Video Resource
